In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

df = pd.read_excel(r"e:\users\shatrov_eo\Desktop\Данные для ML.xlsx")
df_2025 = pd.read_excel(r"e:\users\shatrov_eo\Desktop\Данные предикт 2025.xlsx")

df = df[df['year'] < 2025]  # только данные для обучения

known_features_2025 = [
    'supplier_code', 'year', 'month', 'category', 'group',
    'type_of_compensation', 'pul', 'dz', 'new', 'inf'
]

missing_features = [
    'postavka', 'postavka_1pal', 'revenue', 'revenue_1pal',
    'cost_transp', 'cost_transp_1pal', 'cost_sklad', 'cost_sklad_1pal',
    'sku_count', 'avg_sku_price'
]

target_column = 'effect'
binary_cols = ['pul', 'dz', 'new']
df[binary_cols] = df[binary_cols].astype(int)
df_2025[binary_cols] = df_2025[binary_cols].astype(int)

categorical_cols = ['supplier_code', 'category', 'group', 'type_of_compensation']

def encode_shared(df_train, df_predict, cat_cols):
    encoders = {}
    for col in cat_cols:
        le = LabelEncoder()
        df_train[col] = le.fit_transform(df_train[col].astype(str))
        known_values = set(le.classes_)
        df_predict = df_predict[df_predict[col].astype(str).isin(known_values)]
        df_predict[col] = le.transform(df_predict[col].astype(str))
        encoders[col] = le
    return df_predict

df_2025 = encode_shared(df, df_2025, categorical_cols)

def add_features(data):
    data['quarter'] = ((data['month'] - 1) // 3 + 1).astype(int)
    data['pul_dz_interaction'] = data['pul'] * data['dz']
    return data

df = add_features(df)
df_2025 = add_features(df_2025)

def add_group_stats(data, ref_data, features, group_cols=['supplier_code', 'group']):
    for feature in features:
        for grp_col in group_cols:
            mean_dict = ref_data.groupby(grp_col)[feature].mean()
            col_name = f'{feature}_{grp_col}_mean'
            data[col_name] = data[grp_col].map(mean_dict).fillna(0)
    return data

df = add_group_stats(df, df, [target_column] + missing_features)
df_2025 = add_group_stats(df_2025, df, [target_column] + missing_features)

extended_known_features = known_features_2025 + [
    'quarter', 'pul_dz_interaction'
]
for feature in [target_column] + missing_features:
    for grp in ['supplier_code', 'group']:
        extended_known_features.append(f'{feature}_{grp}_mean')

# Обучение модели LightGBM с RandomizedSearch 
def train_lgb_with_hyperopt(X, y, n_iter=30, random_state=42):
    param_dist = {
        'num_leaves': np.arange(20, 150, 10),
        'max_depth': np.arange(3, 15, 1),
        'learning_rate': np.linspace(0.01, 0.3, 30),
        'n_estimators': np.arange(50, 300, 50),
        'min_child_samples': np.arange(5, 50, 5),
        'subsample': np.linspace(0.5, 1.0, 6),
        'colsample_bytree': np.linspace(0.5, 1.0, 6),
        'reg_alpha': np.linspace(0, 1, 10),
        'reg_lambda': np.linspace(0, 1, 10)
    }

    model = lgb.LGBMRegressor(random_state=random_state, verbosity=-1)

    rand_search = RandomizedSearchCV(
        model, param_distributions=param_dist,
        n_iter=n_iter, cv=3, scoring='neg_mean_squared_error',
        verbose=1, random_state=random_state, n_jobs=-1
    )

    rand_search.fit(X, y)
    print("Лучшие параметры:", rand_search.best_params_)
    return rand_search.best_estimator_

# Предсказание отсутствующих признаков 
predicted_features_2025 = df_2025[known_features_2025].copy()

for feature in missing_features:
    print(f"▶ Обучение модели для {feature}...")
    X_train = df[extended_known_features]
    y_train = df[feature]

    model = train_lgb_with_hyperopt(X_train, y_train, n_iter=25)
    preds = model.predict(df_2025[extended_known_features])
    predicted_features_2025[feature] = pd.Series(preds, index=df_2025.index)

    print(f"✅ {feature}: min={preds.min():.2f}, max={preds.max():.2f}, NaN={pd.isna(preds).sum()}")

# Дополнительные признаки в predicted_features_2025
predicted_features_2025 = add_features(predicted_features_2025)
predicted_features_2025 = add_group_stats(predicted_features_2025, df, [target_column] + missing_features)

full_features_2024 = df[extended_known_features + missing_features]
target_2024 = df[target_column]
full_features_2025 = predicted_features_2025.copy()

X_train, X_val, y_train, y_val = train_test_split(
    full_features_2024, target_2024, test_size=0.2, random_state=42
)

print("▶ Обучение модели для effect (валидация)...")
effect_model_val = train_lgb_with_hyperopt(X_train, y_train, n_iter=30)
val_preds = effect_model_val.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, val_preds))
print(f"📉 RMSE на валидации: {rmse:.4f}")

# Финальное обучение на всех данных
print("▶ Финальное обучение модели для effect на всех данных...")
effect_model = train_lgb_with_hyperopt(full_features_2024, target_2024, n_iter=30)

# Предсказание на 2025
df_2025['predicted_effect'] = effect_model.predict(full_features_2025)

df_2025.to_excel(r"e:\users\shatrov_eo\Desktop\предикт 2025.xlsx", index=False)
print('✅Файл создан')

e:\users\shatrov_eo\AppData\Local\Temp\ipykernel_23144\4185527183.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predict[col] = le.transform(df_predict[col].astype(str))


▶ Обучение модели для postavka...
Fitting 3 folds for each of 25 candidates, totalling 75 fits
Лучшие параметры: {'subsample': np.float64(0.8), 'reg_lambda': np.float64(0.1111111111111111), 'reg_alpha': np.float64(0.6666666666666666), 'num_leaves': np.int64(120), 'n_estimators': np.int64(200), 'min_child_samples': np.int64(25), 'max_depth': np.int64(12), 'learning_rate': np.float64(0.01), 'colsample_bytree': np.float64(0.5)}
✅ postavka: min=-16477538.62, max=1154459241.99, NaN=0
▶ Обучение модели для postavka_1pal...
Fitting 3 folds for each of 25 candidates, totalling 75 fits
Лучшие параметры: {'subsample': np.float64(0.9), 'reg_lambda': np.float64(0.8888888888888888), 'reg_alpha': np.float64(0.3333333333333333), 'num_leaves': np.int64(100), 'n_estimators': np.int64(250), 'min_child_samples': np.int64(25), 'max_depth': np.int64(12), 'learning_rate': np.float64(0.3), 'colsample_bytree': np.float64(0.8)}
✅ postavka_1pal: min=-3131.29, max=62489.31, NaN=0
▶ Обучение модели для revenue...